In [ ]:
import pandas as pd
import numpy as np
import infostop

In [ ]:
#RUN INFOSTOP
def run_infostop(data):

    fitter = infostop.Infostop(r1 = R, 
                               r2 = R, 
                               min_spacial_resolution=0.00001,
                               min_staying_time=10*60,
                               max_time_between = 60*6*60,
                               verbose = True)
    return fitter.fit_predict(data)

def find_labels(data, R = 20):
    

    labels = run_infostop(data)
        
    #POST PROCESS
    users = range(len(data))
    intervals = [post_process_user(n,u,labels[n]) for n,u in (enumerate(data))]
    intervals_global = pd.concat(intervals)
    
    return intervals_global


def post_process_user(n, item, labels):
    r = pd.DataFrame(infostop.postprocess.compute_intervals(item, labels),columns = ['loc','start','end','lat','lon'])
    r["user"] = n
    return r.dropna(subset = ['lat','lon'])

In [ ]:
data = pd.read_csv('data_one_month.csv')
data = data.drop('Unnamed: 0', axis=1)
data

In [ ]:
R = 20
new = []
for key, group in data.groupby('user'):
    g = group.sort_values(by = 'timestamp').copy()
    new.append(g[['lat','lon','timestamp']].values)

In [ ]:
stops = find_labels(new)

In [ ]:
stops.to_csv('stops.csv', index=False)